In [1]:
import h2o
from IPython.display import display
from IPython.core.display import HTML
from cec.constraints_tabular_explainer import ConstraintsTabularExplainer
from cec.data_info import DataInfo

display(HTML("<style>.container { width:98% !important; }</style>"))

h2o.no_progress()
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 23 hours 15 mins
H2O cluster timezone:,Asia/Jerusalem
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.4
H2O cluster version age:,6 months and 25 days !!!
H2O cluster name:,nfrost
H2O cluster total nodes:,1
H2O cluster free memory:,1.515 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


# Load Data

In [2]:
df = h2o.import_file(path="data/bank-full.csv", col_types={'age': 'real'})
col_names = df.col_names.copy()
col_names.remove('y')
data_info = DataInfo(df, col_names)

df.show()

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


# Train Random Forest Model

In [3]:
ntrees = 10
max_depth = 6
seed = 16

In [4]:
train, test = df.split_frame(ratios=[0.8], seed=seed)

model = h2o.estimators.H2ORandomForestEstimator(ntrees=ntrees,
                                                max_depth=max_depth,
                                                seed=seed)
model.train(training_frame=train, validation_frame=test, y='y')
print(model)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1538478431987_908


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.07029844760860517
RMSE: 0.26513854417757743
LogLoss: 0.2378266203492692
Mean Per-Class Error: 0.16555153389480481
AUC: 0.8983809117798257
Gini: 0.7967618235596514
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21165226709506718: 


,no,yes,Error,Rate
no,28623.0,3056.0,0.0965,(3056.0/31679.0)
yes,1227.0,2962.0,0.2929,(1227.0/4189.0)
Total,29850.0,6018.0,0.1194,(4283.0/35868.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2116523,0.5803860,228.0
max f2,0.1144936,0.6841594,283.0
max f0point5,0.3608820,0.5645208,157.0
max accuracy,0.4016028,0.9011933,140.0
max precision,1.0,1.0,0.0
max recall,0.0025283,1.0,399.0
max specificity,1.0,1.0,0.0
max absolute_mcc,0.2057382,0.5252420,231.0
max min_per_class_accuracy,0.1178713,0.8319707,281.0
max mean_per_class_accuracy,0.1113216,0.8344485,285.0


Gains/Lift Table: Avg response rate: 11.66 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100267,0.7373333,6.4615962,6.4615962,0.7666667,0.7666667,0.0647887,0.0647887,546.1596244,546.1596244
,2,0.0200535,0.6295875,5.9933646,6.2274804,0.7111111,0.7388889,0.0600939,0.1248826,499.3364632,522.7480438
,3,0.0300802,0.5215604,5.3378404,5.9309338,0.6333333,0.7037037,0.0535211,0.1784038,433.7840376,493.0933751
,4,0.0401070,0.4717183,4.4950235,5.5719562,0.5333333,0.6611111,0.0450704,0.2234742,349.5023474,457.1956182
,5,0.0501337,0.4462583,4.6823161,5.3940282,0.5555556,0.64,0.0469484,0.2704225,368.2316119,439.4028169
,6,0.1000446,0.3614074,4.2705231,4.8335268,0.5066964,0.5734967,0.2131455,0.4835681,327.0523139,383.3526773
,7,0.1500668,0.2837657,3.2286082,4.2985539,0.3830735,0.5100223,0.1615023,0.6450704,222.8608175,329.8553907
,8,0.2000891,0.1858287,2.1962044,3.7729665,0.2605791,0.4476615,0.1098592,0.7549296,119.6204398,277.2966530
,9,0.3025847,0.0976019,1.2184201,2.9076562,0.1445652,0.3449926,0.1248826,0.8798122,21.8420086,190.7656247
,10,0.4000668,0.0647335,0.6549891,2.3587608,0.0777143,0.2798663,0.0638498,0.9436620,-34.5010865,135.8760752



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-10-04 13:22:23,0.002 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-10-04 13:22:23,0.025 sec,1.0,0.2700802,0.2740974,0.8599398,5.8537208,0.1119420,0.2787771,0.2894118,0.8545254,5.5361502,0.1227718
,2018-10-04 13:22:23,0.040 sec,2.0,0.2798205,0.2985988,0.8162902,6.2758561,0.1234756,0.2777447,0.2628288,0.8663571,5.9247525,0.1368093
,2018-10-04 13:22:23,0.058 sec,3.0,0.2757173,0.2848794,0.8524391,6.2624404,0.1244190,0.2740459,0.2542580,0.8870406,5.9733625,0.1297906
,2018-10-04 13:22:23,0.080 sec,4.0,0.2713442,0.2642152,0.8729642,6.4514952,0.1211396,0.2731703,0.2502782,0.8913613,5.8060720,0.1256684
,2018-10-04 13:22:23,0.109 sec,5.0,0.2697057,0.2542987,0.8807324,6.7587093,0.1267057,0.2725746,0.2481084,0.8933885,5.7124257,0.1292335
,2018-10-04 13:22:23,0.145 sec,6.0,0.2679507,0.2464365,0.8876784,6.8108422,0.1183321,0.2708154,0.2449446,0.8961285,6.1127380,0.1282308
,2018-10-04 13:22:23,0.185 sec,7.0,0.2665592,0.2432145,0.8897154,6.9477641,0.1185964,0.2696219,0.2432489,0.8981294,6.1806573,0.1149733
,2018-10-04 13:22:23,0.231 sec,8.0,0.2670331,0.2430109,0.8903844,6.9713959,0.1235154,0.2699451,0.2444798,0.8999408,6.4832069,0.1154189
,2018-10-04 13:22:23,0.282 sec,9.0,0.2661253,0.2409941,0.8949451,7.1131870,0.1209038,0.2694425,0.2431310,0.9030032,6.2743036,0.1117424


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
duration,3515.1044922,1.0,0.4377236
poutcome,1987.8806152,0.5655253,0.2475438
month,1320.8863525,0.3757744,0.1644853
pdays,251.6174927,0.0715818,0.0313330
contact,184.7824402,0.0525681,0.0230103
job,161.1762543,0.0458525,0.0200707
housing,148.1796417,0.0421551,0.0184523
age,144.6105957,0.0411398,0.0180078
previous,104.5439911,0.0297414,0.0130185
day,63.7111740,0.0181250,0.0079337


# No Constraints Explanation

In [5]:
simple_explainer = ConstraintsTabularExplainer(data_info, 10)

In [6]:
simple_explainer.get_changes(model, test, prediction='no', wanted_tag='yes', seed=seed)

,,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,predict
Error,36,57,admin.,single,secondary,no,5768,yes,no,unknown,19,jun,512,1,-1,0,unknown,yes,no
Changed,36,66,admin.,single,secondary,no,5768,no,no,unknown,19,mar,1049,1,-1,0,success,yes,yes


# Constraints based Explanation

In [7]:
def actionable_constraints(xs):
    x1 = xs[0]
    x2 = xs[1]

    for col in col_names:
        if not col in ['contact', 'month', 'day']:
            if not (x1[col].values == x2[col].values).all():
                return False
    return True

cec_explainer = ConstraintsTabularExplainer(data_info, 10, constraints=actionable_constraints)

In [8]:
cec_explainer.get_changes(model, test, prediction='no', wanted_tag='yes', seed=seed)

,,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,predict
Error,36,57,admin.,single,secondary,no,5768,yes,no,unknown,19,jun,512,1,-1,0,unknown,yes,no
Changed,36,57,admin.,single,secondary,no,5768,yes,no,cellular,31,oct,512,1,-1,0,unknown,yes,yes
